In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
#게시판 메뉴 수동으로 누르게 수정
#페이징 수정
#이메일아이디 수정 

# pip install pymysql selenium beautifulsoup4 lxml openpyxl pandas

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import BIGINT, VARCHAR, TEXT
from sqlalchemy.dialects.mysql import LONGTEXT

import time
import re
import traceback
import pandas as pd
import json
from datetime import datetime

# BASE URL 상단 정의
BASE = "http://office.anyfive.com/home/"

# ─────────────────────────────────────────────────────────
# BeautifulSoup parser fallback
# ─────────────────────────────────────────────────────────
try:
    import lxml  # noqa: F401
    BS_PARSER = "lxml"
except Exception:
    BS_PARSER = "html.parser"

# ─────────────────────────────────────────────────────────
# 1) 공통 유틸
# ─────────────────────────────────────────────────────────
def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    # 최상위
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if text_contains:
        els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
        for el in els:
            if text_contains in (el.text or ""):
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                driver.execute_script("arguments[0].click();", el)
                return True
    # 프레임 내부
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if text_contains:
            els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
            for el in els:
                if text_contains in (el.text or ""):
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    driver.execute_script("arguments[0].click();", el)
                    return True
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css):
    """selector_css 요소가 보이는 프레임으로 전환"""
    driver.switch_to.default_content()
    if driver.find_elements(By.CSS_SELECTOR, selector_css): 
        return True
    for fr in driver.find_elements(By.CSS_SELECTOR, "iframe, frame"):
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if driver.find_elements(By.CSS_SELECTOR, selector_css): 
            return True
    driver.switch_to.default_content()
    return False

NBSP = "\xa0"

def norm_space(s: str) -> str:
    if not s: return ""
    s = s.replace(NBSP, " ")
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def datetime_to_timestamp(dt_str):
    """날짜 문자열을 밀리초 timestamp로 변환"""
    try:
        dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S")
        return str(int(dt.timestamp() * 1000))
    except:
        return ""

# ─────────────────────────────────────────────────────────
# 2) CSV에서 사용자 정보 로드 (emailId 매핑용)
# ─────────────────────────────────────────────────────────
def load_user_mapping(csv_path):
    """CSV에서 한글이름/부서명 → 로그인ID 매핑"""
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    user_map = {}
    for _, row in df.iterrows():
        key = f"{row['한글이름']}/{row['부서명']}"
        user_map[key] = row['로그인 ID']
    return user_map

# ─────────────────────────────────────────────────────────
# 3) 게시판 목록 수집 함수
# ─────────────────────────────────────────────────────────
def has_next_page_board():
    """다음 페이지가 있는지 확인 (프레임 내부)"""
    try:
        # 1단계: CSS 셀렉터로 시도
        selectors = [
            "a.k-pager-nav:not(.k-state-disabled) span.k-i-arrow-e",  # 다음 화살표 아이콘
            "a.k-pager-nav[aria-label*='next']:not(.k-state-disabled)",
            "a.k-pager-nav[aria-label*='다음']:not(.k-state-disabled)",
        ]
        
        for selector in selectors:
            next_btns = driver.find_elements(By.CSS_SELECTOR, selector)
            if next_btns:
                print(f"  ✓ 다음 페이지 버튼 발견: {selector}")
                return True
        
        print("  ℹ️ 다음 페이지 버튼 없음 - 마지막 페이지")
        return False
        
    except Exception as e:
        print(f"  ⚠️ 페이징 확인 실패: {e}")
        return False

def go_to_next_page_board():
    """다음 페이지로 이동 (프레임 내부)"""
    try:
        # 동일한 방식으로 버튼 찾기
        selectors = [
            "a.k-pager-nav:not(.k-state-disabled) span.k-i-arrow-e",  # 화살표 아이콘의 부모
            "a.k-pager-nav[aria-label*='next']:not(.k-state-disabled)",
            "a.k-pager-nav[aria-label*='다음']:not(.k-state-disabled)",
        ]
        
        next_btn = None
        for selector in selectors:
            if "span.k-i-arrow-e" in selector:
                # 화살표 아이콘으로 찾는 경우, 부모 a 태그 선택
                icons = driver.find_elements(By.CSS_SELECTOR, "span.k-i-arrow-e")
                for icon in icons:
                    parent = icon.find_element(By.XPATH, "..")
                    if "k-pager-nav" in parent.get_attribute("class") and "k-state-disabled" not in parent.get_attribute("class"):
                        next_btn = parent
                        print(f"  📄 다음 페이지로 이동 중... (아이콘으로 발견)")
                        break
            else:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements:
                    next_btn = elements[0]
                    print(f"  📄 다음 페이지로 이동 중... (셀렉터: {selector})")
                    break
            
            if next_btn:
                break
        
        if not next_btn:
            print("  ❌ 다음 페이지 버튼을 찾을 수 없습니다")
            return False
        
        # 클릭
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", next_btn)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", next_btn)
        
        # 로딩 대기
        time.sleep(3)
        
        # 로딩 마스크 대기
        try:
            WebDriverWait(driver, 15).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
            )
        except:
            pass
        
        # 그리드 재로딩 대기
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.grdTit"))
            )
        except:
            pass
        
        print("  ✅ 페이지 이동 완료")
        return True
        
    except Exception as e:
        print(f"  ❌ 페이지 이동 실패: {e}")
        traceback.print_exc()
        return False
# ─────────────────────────────────────────────────────────
# 4) 게시판 상세 크롤링 함수
# ─────────────────────────────────────────────────────────
def parse_board_detail(driver, user_map):
    """
    게시판 상세 페이지 파싱 → 다우 JSON 형식으로 변환
    """
    # content_frame으로 전환 (게시판은 프레임 구조)
    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("content_frame")
        print("  ✓ content_frame 전환 완료")
    except Exception as e:
        print(f"  ⚠️ 프레임 전환 실패: {e}")
        pass
    
    # DOM 준비 대기
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.tbl_none-titpoint"))
        )
        print("  ✓ 상세 페이지 로딩 완료")
    except Exception as e:
        print(f"  ⚠️ 상세 페이지 로딩 실패: {e}")
        return None
    
    soup = BeautifulSoup(driver.page_source, BS_PARSER)
    
    # 1) 기본 정보 추출
    atcl_seq = soup.select_one("input[name='atcl_seq']")
    atcl_seq_val = atcl_seq.get('value', '') if atcl_seq else ""
    
    brd_seq = soup.select_one("input[name='brd_seq']")
    brd_seq_val = brd_seq.get('value', '') if brd_seq else ""
    
    title_el = soup.select_one("table.tbl_none-titpoint td")
    title = norm_space(title_el.get_text()) if title_el else ""
    
    # 2) 작성자 정보 (한글이름/부서/직위)
    writer_el = soup.select_one("th:-soup-contains('최초 작성자') + td span.abcUsr")
    if not writer_el:
        writer_el = soup.select_one("span.abcUsr[data]")
    
    writer_text = norm_space(writer_el.get_text()) if writer_el else ""
    writer_usr_key = writer_el.get('data', '') if writer_el else ""
    
    # CSV에서 emailId 찾기 (한글이름으로 매칭)
    email_id = ""
    if writer_text:
        # "경영기획팀/기타/공통부문" 형태에서 첫 부분이 한글이름일 수 있음
        # 또는 "유수영/시스템사업팀/이사" 같은 형태
        parts = writer_text.split("/")
        
        # 첫 부분으로 CSV에서 검색 (한글이름 또는 부서명)
        search_name = parts[0].strip() if parts else ""
        
        # 1차: 한글이름으로 정확 매칭
        for key, login_id in user_map.items():
            name_in_key = key.split("/")[0] if "/" in key else key
            if search_name == name_in_key:
                email_id = login_id
                break
        
        # 2차: 부서명으로 매칭 (여러 명 있을 경우 첫 번째)
        if not email_id and len(parts) >= 2:
            dept_name = parts[1].strip() if len(parts) > 1 else parts[0].strip()
            for key, login_id in user_map.items():
                if dept_name in key:
                    email_id = login_id
                    break
    
    # usr_key를 emailId로 사용 (매칭 실패 시)
    # if not email_id:
    #     email_id = writer_usr_key
    
    # 3) 작성일
    date_el = soup.select_one("th:-soup-contains('작성일') + td span.date")
    created_at = norm_space(date_el.get_text()) if date_el else ""
    created_timestamp = datetime_to_timestamp(created_at)
    
    # 4) 조회수
    read_count_el = soup.select_one("th:-soup-contains('조회자') + td span.cnt_n")
    read_count = norm_space(read_count_el.get_text()) if read_count_el else "0"
    
    # 5) 본문 추출
    content_div = soup.select_one("div#atcl_cont")
    content_html = ""
    if content_div:
        content_html = str(content_div)
    
    # 6) 첨부파일 추출
    attaches = []
    attach_area = soup.select_one("th.vertTop:-soup-contains('첨부파일') + td")
    if attach_area:
        for li in attach_area.select("li"):
            onclick = li.select_one("span[onclick*='fileDownLoad']")
            if onclick:
                onclick_attr = onclick.get('onclick', '')
                # fileDownLoad('brd/20250918/...')에서 경로 추출
                path_match = re.search(r"fileDownLoad\('([^']+)'\)", onclick_attr)
                file_path = path_match.group(1) if path_match else ""
                
                file_name_el = li.select_one("span.verti_Top")
                file_name = norm_space(file_name_el.get_text()) if file_name_el else ""
                
                if file_name and file_path:
                    attaches.append({
                        "path": f"file:///{file_path}",
                        "name": file_name
                    })

    # 7) 댓글 추출
    comments = []
    for article in soup.select("article.comentWrap"):
        cmt_writer_el = article.select_one("span.abcUsr")
        cmt_writer_text = norm_space(cmt_writer_el.get_text()) if cmt_writer_el else ""
        cmt_writer_usr_key = cmt_writer_el.get('data', '') if cmt_writer_el else ""
        
        # 댓글 작성자도 CSV 매칭
        cmt_email_id = ""
        if cmt_writer_text:
            parts = cmt_writer_text.split("/")
            search_name = parts[0].strip() if parts else ""
            
            # 1차: 한글이름 정확 매칭
            for key, login_id in user_map.items():
                name_in_key = key.split("/")[0] if "/" in key else key
                if search_name == name_in_key:
                    cmt_email_id = login_id
                    break
            
            # 2차: 부분 매칭
            if not cmt_email_id:
                for key, login_id in user_map.items():
                    if search_name in key:
                        cmt_email_id = login_id
                        break
        # if not cmt_email_id:
        #     cmt_email_id = cmt_writer_usr_key
        
        cmt_text_el = article.select_one("li.text pre")
        cmt_text = norm_space(cmt_text_el.get_text()) if cmt_text_el else ""
        
        cmt_date_el = article.select_one("li.date_time")
        cmt_date = norm_space(cmt_date_el.get_text()) if cmt_date_el else ""
        cmt_timestamp = datetime_to_timestamp(cmt_date)
        
        # ✅ 유효성 검증: 메시지나 날짜가 있는 댓글만 추가
        if cmt_text or cmt_timestamp:  # 메시지나 작성일이 있으면 실제 댓글
            comments.append({
                "sourceId": f"{brd_seq_val}_{atcl_seq_val}_{len(comments)}",
                "sourcePostId": f"{brd_seq_val}_{atcl_seq_val}",
                "createdAt": cmt_timestamp,
                "writer": {"emailId": cmt_email_id},
                "message": cmt_text,
                "attaches": ""
            })
    
    # 8) 다우 JSON 형식으로 변환
    post_data = {
        "sourceId": f"{brd_seq_val}_{atcl_seq_val}",
        "targetBoardId": brd_seq_val,
        "createdAt": created_timestamp,
        "updatedAt": created_timestamp,  # 수정일이 없으면 작성일과 동일
        "inlineHandling": "True",
        "writer": {"emailId": email_id},
        "title": title,
        "readCount": read_count,
        "content": content_html,
        "status": "OPEN",
        "attaches": attaches
    }
    
    return {
        "post": post_data,
        "comments": comments,
        "atcl_seq": atcl_seq_val,
        "brd_seq": brd_seq_val
    }

# ─────────────────────────────────────────────────────────
# 5) 게시판 행 클릭하여 상세 열기
# ─────────────────────────────────────────────────────────
def click_board_row(tr):
    """게시판 행 클릭하여 상세 열기"""
    try:
        # a.grdTit 링크 클릭
        title_link = tr.find_element(By.CSS_SELECTOR, "a.grdTit")
        
        # onclick 속성에서 atcl_seq 추출
        onclick_attr = title_link.get_attribute("onclick") or ""
        # "brdAtclList.showGo(this,4099032,true)" → 4099032 추출
        atcl_seq_match = re.search(r'showGo\([^,]+,(\d+)', onclick_attr)
        if atcl_seq_match:
            atcl_seq = atcl_seq_match.group(1)
            print(f"  📌 게시글 ID: {atcl_seq}")
        
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", title_link)
        driver.execute_script("arguments[0].click();", title_link)
        time.sleep(2)
        return True
    except Exception as e:
        print(f"  ❌ 클릭 실패: {e}")
        try:
            # 더블클릭 시도
            ActionChains(driver).double_click(tr).perform()
            time.sleep(1.5)
            return True
        except:
            return False

def extract_row_info(tr):
    """목록 행에서 기본 정보 추출"""
    try:
        td = tr.find_element(By.CSS_SELECTOR, "td:has(a.grdTit)")
        
        # 게시판명 추출 (예: [전사게시판])
        board_name_el = td.find_elements(By.CSS_SELECTOR, "span[style*='color:rgb(83,193,75)']")
        board_name = board_name_el[0].text.strip() if board_name_el else ""
        
        # 제목 추출
        title_link = td.find_element(By.CSS_SELECTOR, "a.grdTit")
        title = norm_space(title_link.text)
        
        # onclick에서 atcl_seq 추출
        onclick_attr = title_link.get_attribute("onclick") or ""
        atcl_seq_match = re.search(r'showGo\([^,]+,(\d+)', onclick_attr)
        atcl_seq = atcl_seq_match.group(1) if atcl_seq_match else ""
        
        # NEW 아이콘 확인
        has_new = len(td.find_elements(By.CSS_SELECTOR, "span.o-i-new")) > 0
        
        # 댓글 수 추출
        comment_count_el = td.find_elements(By.CSS_SELECTOR, "span.F_11_gray[style*='color:red']")
        comment_count = norm_space(comment_count_el[0].text) if comment_count_el else "0"
        
        return {
            "atcl_seq": atcl_seq,
            "board_name": board_name,
            "title": title,
            "has_new": has_new,
            "comment_count": comment_count
        }
    except Exception as e:
        print(f"  ⚠️ 목록 정보 추출 실패: {e}")
        return None

def back_to_board_list():
    """상세에서 목록으로 복귀"""
    try:
        driver.switch_to.default_content()
        
        # content_frame으로 이동
        try:
            driver.switch_to.frame("content_frame")
        except:
            pass
        
        # 목록 버튼 클릭
        list_btn = driver.find_elements(By.XPATH, 
            "//button[contains(text(),'목록')] | //a[contains(text(),'목록')] | //span[contains(text(),'목록')]")
        if list_btn:
            driver.execute_script("arguments[0].click();", list_btn[0])
            time.sleep(1.5)
        
        driver.switch_to.default_content()
        return True
    except Exception as e:
        print(f"⚠️ 목록 복귀 실패: {e}")
        driver.switch_to.default_content()
        return False

# ─────────────────────────────────────────────────────────
# 6) 메인 실행 로직
# ─────────────────────────────────────────────────────────

# 브라우저 시작
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-popup-blocking")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(service=Service(), options=opts)

# 로그인
driver.get(BASE)
time.sleep(1.2)
input("로그인(필요시 '다음에 변경')까지 완료 후 Enter... ")

# 사용자 매핑 로드
print("📂 사용자 정보 로딩 중...")
user_map = load_user_mapping("users_filtered.csv")
print(f"✅ {len(user_map)}명의 사용자 정보 로드 완료")

# ✨ 게시판 수동 선택 대기 (수정된 부분)
print("\n" + "="*70)
print("📋 원하는 게시판으로 이동해주세요")
print("="*70)
print("1. 좌측 메뉴에서 '게시판' 클릭")
print("2. '(주)애니파이브' 하위 메뉴 확장")
print("3. 원하는 상세 게시판 선택")
print("4. 게시판 목록이 화면에 표시되면 Enter를 눌러주세요")
print("="*70)
input("준비 완료 후 Enter... ")

print("\n📊 크롤링 시작...")
time.sleep(1)

# content_frame으로 전환
try:
    driver.switch_to.default_content()
    driver.switch_to.frame("content_frame")
except:
    pass

# 게시판 목록 수집
all_posts = []
all_comments = []
page_num = 1
max_pages = 999  # 테스트용
max_test = 50   # 페이지당 최대 수집 수

# 그리드 테이블 찾기 (a.grdTit가 있는 테이블)
GRID_SELECTOR = "table"  # 먼저 table을 찾고 a.grdTit 확인

while page_num <= max_pages:
    print(f"\n{'='*70}")
    print(f"📄 페이지 {page_num} 수집 중...")
    print(f"{'='*70}")
    
    try:
        # 프레임 재진입
        driver.switch_to.default_content()
        driver.switch_to.frame("content_frame")
        
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.grdTit"))
        )
        
        # a.grdTit가 있는 행만 수집
        rows = driver.find_elements(By.CSS_SELECTOR, "tr:has(a.grdTit)")
        
        if not rows:
            # :has() 미지원 브라우저 대응
            all_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr, tr")
            rows = [r for r in all_rows if r.find_elements(By.CSS_SELECTOR, "a.grdTit")]
        
        print(f"  ✅ 게시글 {len(rows)}개 발견")
        
        # 페이지당 일부만 수집
        for idx in range(min(max_test, len(rows))):
            try:
                # 매번 rows 재조회 (Stale Element 방지)
                driver.switch_to.default_content()
                driver.switch_to.frame("content_frame")
                
                # a.grdTit가 있는 행만 수집
                current_rows = driver.find_elements(By.CSS_SELECTOR, "tr:has(a.grdTit)")
                if not current_rows:
                    all_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr, tr")
                    current_rows = [r for r in all_rows if r.find_elements(By.CSS_SELECTOR, "a.grdTit")]
                
                if idx >= len(current_rows):
                    print(f"⚠️ 행이 부족합니다 ({idx+1} > {len(current_rows)})")
                    break
                
                tr = current_rows[idx]
                
                # 목록 정보 추출
                row_info = extract_row_info(tr)
                if row_info:
                    print(f"\n📄 [{idx+1}/{min(max_test, len(rows))}] {row_info['title'][:50]}")
                    print(f"   게시판: {row_info['board_name']}, 댓글: {row_info['comment_count']}개")
                else:
                    print(f"\n📄 [{idx+1}/{min(max_test, len(rows))}] 처리 중...")
                
                # 행 클릭하여 상세 열기
                if not click_board_row(tr):
                    print("❌ 상세 열기 실패")
                    continue
                
                # 상세 파싱
                detail_data = parse_board_detail(driver, user_map)
                
                if not detail_data:
                    print("❌ 상세 파싱 실패")
                    back_to_board_list()
                    continue
                
                all_posts.append(detail_data["post"])
                all_comments.extend(detail_data["comments"])
                
                print(f"✅ 수집 완료: {detail_data['post']['title'][:50]}")
                print(f"   - 댓글 {len(detail_data['comments'])}개")
                
                # 목록 복귀
                back_to_board_list()
                time.sleep(1)
                
                # 프레임 재진입 (목록으로 돌아온 후)
                driver.switch_to.default_content()
                driver.switch_to.frame("content_frame")
                
                # 그리드 재로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.grdTit"))
                )
                
            except Exception as e:
                print(f"❌ 수집 실패: {e}")
                traceback.print_exc()
                back_to_board_list()
                continue
        
        print(f"✅ 페이지 {page_num} 완료: {min(max_test, len(rows))}건 시도")
        
        # 프레임 컨텍스트 유지한 채로 페이징 확인
        if has_next_page_board():
            if go_to_next_page_board():
                page_num += 1
                time.sleep(2)
            else:
                print("❌ 페이지 이동 실패 - 종료")
                break
        else:
            print(f"✅ 마지막 페이지 도달")
            break
            
    except Exception as e:
        print(f"❌ 페이지 수집 실패: {e}")
        traceback.print_exc()
        break

# ─────────────────────────────────────────────────────────
# 7) 결과 저장
# ─────────────────────────────────────────────────────────
print(f"\n{'='*70}")
print(f"📊 수집 완료")
print(f"{'='*70}")
print(f"✅ 게시글: {len(all_posts)}건")
print(f"✅ 댓글: {len(all_comments)}건")

# JSON 저장
posts_json = {"posts": all_posts}
comments_json = {"comments": all_comments}

with open("board_posts.json", "w", encoding="utf-8") as f:
    json.dump(posts_json, f, ensure_ascii=False, indent=2)

with open("board_comments.json", "w", encoding="utf-8") as f:
    json.dump(comments_json, f, ensure_ascii=False, indent=2)

print("✅ JSON 저장 완료: board_posts.json, board_comments.json")

# DataFrame 변환 (DB 저장용)
if all_posts:
    df_posts = pd.DataFrame(all_posts)
    df_posts['writer_emailId'] = df_posts['writer'].apply(lambda x: x.get('emailId', ''))
    df_posts['attaches_json'] = df_posts['attaches'].apply(lambda x: json.dumps(x, ensure_ascii=False))
    df_posts = df_posts.drop(['writer', 'attaches'], axis=1)
    
    # Excel 저장
    df_posts.to_excel("게시판_목록.xlsx", index=False, engine="openpyxl")
    df_posts.to_csv("게시판_목록.csv", index=False, encoding="utf-8-sig")
    print("✅ 파일 저장 완료: 게시판_목록.xlsx / 게시판_목록.csv")
    
    # DB 저장
    USER = "root"
    PASS = "1234"
    HOST = "127.0.0.1"
    PORT = 3306
    DB = "crawling"
    
    engine = create_engine(f"mysql+pymysql://{USER}:{PASS}@{HOST}:{PORT}/{DB}?charset=utf8mb4")
    
    df_posts.to_sql(
        name="board_posts",
        con=engine,
        if_exists="replace",
        index=False,
        dtype={
            "sourceId": VARCHAR(100),
            "targetBoardId": VARCHAR(50),
            "createdAt": VARCHAR(30),
            "updatedAt": VARCHAR(30),
            "inlineHandling": VARCHAR(10),
            "writer_emailId": VARCHAR(100),
            "title": VARCHAR(500),
            "readCount": VARCHAR(20),
            "content": LONGTEXT(),
            "status": VARCHAR(20),
            "attaches_json": TEXT()
        }
    )
    print("✅ DB 저장 완료 (테이블: board_posts)")

if all_comments:
    df_comments = pd.DataFrame(all_comments)
    df_comments['writer_emailId'] = df_comments['writer'].apply(lambda x: x.get('emailId', ''))
    df_comments = df_comments.drop(['writer'], axis=1)
    
    df_comments.to_excel("게시판_댓글.xlsx", index=False, engine="openpyxl")
    df_comments.to_csv("게시판_댓글.csv", index=False, encoding="utf-8-sig")
    print("✅ 파일 저장 완료: 게시판_댓글.xlsx / 게시판_댓글.csv")
    
    df_comments.to_sql(
        name="board_comments",
        con=engine,
        if_exists="replace",
        index=False,
        dtype={
            "sourceId": VARCHAR(100),
            "sourcePostId": VARCHAR(100),
            "createdAt": VARCHAR(30),
            "writer_emailId": VARCHAR(100),
            "message": TEXT(),
            "attaches": VARCHAR(500)
        }
    )
    print("✅ DB 저장 완료 (테이블: board_comments)")

# driver.quit()
print("\n🎉 모든 작업 완료!")

로그인(필요시 '다음에 변경')까지 완료 후 Enter...  


📂 사용자 정보 로딩 중...
✅ 184명의 사용자 정보 로드 완료

📋 원하는 게시판으로 이동해주세요
1. 좌측 메뉴에서 '게시판' 클릭
2. '(주)애니파이브' 하위 메뉴 확장
3. 원하는 상세 게시판 선택
4. 게시판 목록이 화면에 표시되면 Enter를 눌러주세요


준비 완료 후 Enter...  



📊 크롤링 시작...

📄 페이지 1 수집 중...
  ✅ 게시글 50개 발견

📄 [1/50] 11월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4098793
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 11월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   - 댓글 1개

📄 [2/50] 10월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4087362
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 10월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   - 댓글 2개

📄 [3/50] 9월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4073932
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 9월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   - 댓글 1개

📄 [4/50] 8월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4061151
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 8월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   - 댓글 2개

📄 [5/50] 7월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4046174
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 7월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   - 댓글 2개

📄 [6/50] 6월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: , 댓글: 0개
  📌 게시글 ID: 4034091
  ✓ content_frame 전환 완료
  ✓ 상세 페이지

In [3]:
# pip install pymysql selenium beautifulsoup4 lxml openpyxl pandas

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import BIGINT, VARCHAR, TEXT
from sqlalchemy.dialects.mysql import LONGTEXT

import time
import re
import traceback
import pandas as pd
import json
from datetime import datetime

# BASE URL 상단 정의
BASE = "http://office.anyfive.com/home/"

# ─────────────────────────────────────────────────────────
# BeautifulSoup parser fallback
# ─────────────────────────────────────────────────────────
try:
    import lxml  # noqa: F401
    BS_PARSER = "lxml"
except Exception:
    BS_PARSER = "html.parser"

# ─────────────────────────────────────────────────────────
# 1) 공통 유틸
# ─────────────────────────────────────────────────────────
def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    # 최상위
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if text_contains:
        els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
        for el in els:
            if text_contains in (el.text or ""):
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                driver.execute_script("arguments[0].click();", el)
                return True
    # 프레임 내부
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if text_contains:
            els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
            for el in els:
                if text_contains in (el.text or ""):
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    driver.execute_script("arguments[0].click();", el)
                    return True
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css):
    """selector_css 요소가 보이는 프레임으로 전환"""
    driver.switch_to.default_content()
    if driver.find_elements(By.CSS_SELECTOR, selector_css): 
        return True
    for fr in driver.find_elements(By.CSS_SELECTOR, "iframe, frame"):
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if driver.find_elements(By.CSS_SELECTOR, selector_css): 
            return True
    driver.switch_to.default_content()
    return False

NBSP = "\xa0"

def norm_space(s: str) -> str:
    if not s: return ""
    s = s.replace(NBSP, " ")
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def datetime_to_timestamp(dt_str):
    """날짜 문자열을 밀리초 timestamp로 변환"""
    try:
        dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S")
        return str(int(dt.timestamp() * 1000))
    except:
        return ""

# ─────────────────────────────────────────────────────────
# 2) CSV에서 사용자 정보 로드 (emailId 매핑용)
# ─────────────────────────────────────────────────────────
def load_user_mapping(csv_path):
    """CSV에서 한글이름/부서명 → 로그인ID 매핑"""
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    user_map = {}
    for _, row in df.iterrows():
        key = f"{row['한글이름']}/{row['부서명']}"
        user_map[key] = row['로그인 ID']
    return user_map

# ─────────────────────────────────────────────────────────
# 3) 게시판 목록 수집 함수
# ─────────────────────────────────────────────────────────
def has_next_page_board():
    """다음 페이지가 있는지 확인"""
    try:
        # 게시판은 페이지네이션 구조가 다를 수 있으니 실제 HTML 확인 필요
        next_btn = driver.find_elements(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        return len(next_btn) > 0
    except:
        return False

def go_to_next_page_board():
    """다음 페이지로 이동"""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        
        print(f"📄 다음 페이지로 이동 중...")
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        
        try:
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
            )
        except:
            pass
        
        return True
    except Exception as e:
        print(f"❌ 마지막 페이지 도달")
        return False

# ─────────────────────────────────────────────────────────
# 4) 게시판 상세 크롤링 함수
# ─────────────────────────────────────────────────────────
def parse_board_detail(driver, user_map):
    """
    게시판 상세 페이지 파싱 → 다우 JSON 형식으로 변환
    """
    # content_frame으로 전환 (게시판은 프레임 구조)
    try:
        driver.switch_to.default_content()
        driver.switch_to.frame("content_frame")
        print("  ✓ content_frame 전환 완료")
    except Exception as e:
        print(f"  ⚠️ 프레임 전환 실패: {e}")
        pass
    
    # DOM 준비 대기
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.tbl_none-titpoint"))
        )
        print("  ✓ 상세 페이지 로딩 완료")
    except Exception as e:
        print(f"  ⚠️ 상세 페이지 로딩 실패: {e}")
        return None
    
    soup = BeautifulSoup(driver.page_source, BS_PARSER)
    
    # 1) 기본 정보 추출
    atcl_seq = soup.select_one("input[name='atcl_seq']")
    atcl_seq_val = atcl_seq.get('value', '') if atcl_seq else ""
    
    brd_seq = soup.select_one("input[name='brd_seq']")
    brd_seq_val = brd_seq.get('value', '') if brd_seq else ""
    
    title_el = soup.select_one("table.tbl_none-titpoint td")
    title = norm_space(title_el.get_text()) if title_el else ""
    
    # 2) 작성자 정보 (한글이름/부서/직위)
    writer_el = soup.select_one("th:-soup-contains('최초 작성자') + td span.abcUsr")
    if not writer_el:
        writer_el = soup.select_one("span.abcUsr[data]")
    
    writer_text = norm_space(writer_el.get_text()) if writer_el else ""
    writer_usr_key = writer_el.get('data', '') if writer_el else ""
    
    # CSV에서 emailId 찾기 (한글이름으로 매칭)
    email_id = ""
    if writer_text:
        # "경영기획팀/기타/공통부문" 형태에서 첫 부분이 한글이름일 수 있음
        # 또는 "유수영/시스템사업팀/이사" 같은 형태
        parts = writer_text.split("/")
        
        # 첫 부분으로 CSV에서 검색 (한글이름 또는 부서명)
        search_name = parts[0].strip() if parts else ""
        
        # 1차: 한글이름으로 정확 매칭
        for key, login_id in user_map.items():
            name_in_key = key.split("/")[0] if "/" in key else key
            if search_name == name_in_key:
                email_id = login_id
                break
        
        # 2차: 부서명으로 매칭 (여러 명 있을 경우 첫 번째)
        if not email_id and len(parts) >= 2:
            dept_name = parts[1].strip() if len(parts) > 1 else parts[0].strip()
            for key, login_id in user_map.items():
                if dept_name in key:
                    email_id = login_id
                    break
    
    # usr_key를 emailId로 사용 (매칭 실패 시)
    if not email_id:
        email_id = writer_usr_key
    
    # 3) 작성일
    date_el = soup.select_one("th:-soup-contains('작성일') + td span.date")
    created_at = norm_space(date_el.get_text()) if date_el else ""
    created_timestamp = datetime_to_timestamp(created_at)
    
    # 4) 조회수
    read_count_el = soup.select_one("th:-soup-contains('조회자') + td span.cnt_n")
    read_count = norm_space(read_count_el.get_text()) if read_count_el else "0"
    
    # 5) 본문 추출
    content_div = soup.select_one("div#atcl_cont")
    content_html = ""
    if content_div:
        content_html = str(content_div)
    
    # 6) 첨부파일 추출
    attaches = []
    attach_area = soup.select_one("th.vertTop:-soup-contains('첨부파일') + td")
    if attach_area:
        for li in attach_area.select("li"):
            onclick = li.select_one("span[onclick*='fileDownLoad']")
            if onclick:
                onclick_attr = onclick.get('onclick', '')
                # fileDownLoad('brd/20250918/...')에서 경로 추출
                path_match = re.search(r"fileDownLoad\('([^']+)'\)", onclick_attr)
                file_path = path_match.group(1) if path_match else ""
                
                file_name_el = li.select_one("span.verti_Top")
                file_name = norm_space(file_name_el.get_text()) if file_name_el else ""
                
                if file_name and file_path:
                    attaches.append({
                        "path": f"file:///{file_path}",
                        "name": file_name
                    })

    # # 7) 댓글 추출 수정 전 (빈 값 들어옴)
    # comments = []
    # for article in soup.select("article.comentWrap"):
    #     cmt_writer_el = article.select_one("span.abcUsr")
    #     cmt_writer_text = norm_space(cmt_writer_el.get_text()) if cmt_writer_el else ""
    #     cmt_writer_usr_key = cmt_writer_el.get('data', '') if cmt_writer_el else ""
        
    #     # 댓글 작성자도 CSV 매칭
    #     cmt_email_id = ""
    #     if cmt_writer_text:
    #         parts = cmt_writer_text.split("/")
    #         search_name = parts[0].strip() if parts else ""
            
    #         # 1차: 한글이름 정확 매칭
    #         for key, login_id in user_map.items():
    #             name_in_key = key.split("/")[0] if "/" in key else key
    #             if search_name == name_in_key:
    #                 cmt_email_id = login_id
    #                 break
            
    #         # 2차: 부분 매칭
    #         if not cmt_email_id:
    #             for key, login_id in user_map.items():
    #                 if search_name in key:
    #                     cmt_email_id = login_id
    #                     break
    #     if not cmt_email_id:
    #         cmt_email_id = cmt_writer_usr_key
        
    #     cmt_text_el = article.select_one("li.text pre")
    #     cmt_text = norm_space(cmt_text_el.get_text()) if cmt_text_el else ""
        
    #     cmt_date_el = article.select_one("li.date_time")
    #     cmt_date = norm_space(cmt_date_el.get_text()) if cmt_date_el else ""
    #     cmt_timestamp = datetime_to_timestamp(cmt_date)
        
    #     comments.append({
    #         "sourceId": f"{brd_seq_val}_{atcl_seq_val}_{len(comments)}",
    #         "sourcePostId": f"{brd_seq_val}_{atcl_seq_val}",
    #         "createdAt": cmt_timestamp,
    #         "writer": {"emailId": cmt_email_id},
    #         "message": cmt_text,
    #         "attaches": ""
    #     })


    # 7) 댓글 추출
    comments = []
    for article in soup.select("article.comentWrap"):
        cmt_writer_el = article.select_one("span.abcUsr")
        cmt_writer_text = norm_space(cmt_writer_el.get_text()) if cmt_writer_el else ""
        cmt_writer_usr_key = cmt_writer_el.get('data', '') if cmt_writer_el else ""
        
        # 댓글 작성자도 CSV 매칭
        cmt_email_id = ""
        if cmt_writer_text:
            parts = cmt_writer_text.split("/")
            search_name = parts[0].strip() if parts else ""
            
            # 1차: 한글이름 정확 매칭
            for key, login_id in user_map.items():
                name_in_key = key.split("/")[0] if "/" in key else key
                if search_name == name_in_key:
                    cmt_email_id = login_id
                    break
            
            # 2차: 부분 매칭
            if not cmt_email_id:
                for key, login_id in user_map.items():
                    if search_name in key:
                        cmt_email_id = login_id
                        break
        if not cmt_email_id:
            cmt_email_id = cmt_writer_usr_key
        
        cmt_text_el = article.select_one("li.text pre")
        cmt_text = norm_space(cmt_text_el.get_text()) if cmt_text_el else ""
        
        cmt_date_el = article.select_one("li.date_time")
        cmt_date = norm_space(cmt_date_el.get_text()) if cmt_date_el else ""
        cmt_timestamp = datetime_to_timestamp(cmt_date)
        
        # ✅ 유효성 검증: 메시지나 날짜가 있는 댓글만 추가
        if cmt_text or cmt_timestamp:  # 메시지나 작성일이 있으면 실제 댓글
            comments.append({
                "sourceId": f"{brd_seq_val}_{atcl_seq_val}_{len(comments)}",
                "sourcePostId": f"{brd_seq_val}_{atcl_seq_val}",
                "createdAt": cmt_timestamp,
                "writer": {"emailId": cmt_email_id},
                "message": cmt_text,
                "attaches": ""
            })
    
    # 8) 다우 JSON 형식으로 변환
    post_data = {
        "sourceId": f"{brd_seq_val}_{atcl_seq_val}",
        "targetBoardId": brd_seq_val,
        "createdAt": created_timestamp,
        "updatedAt": created_timestamp,  # 수정일이 없으면 작성일과 동일
        "inlineHandling": "True",
        "writer": {"emailId": email_id},
        "title": title,
        "readCount": read_count,
        "content": content_html,
        "status": "OPEN",
        "attaches": attaches
    }
    
    return {
        "post": post_data,
        "comments": comments,
        "atcl_seq": atcl_seq_val,
        "brd_seq": brd_seq_val
    }

# ─────────────────────────────────────────────────────────
# 5) 게시판 행 클릭하여 상세 열기
# ─────────────────────────────────────────────────────────
def click_board_row(tr):
    """게시판 행 클릭하여 상세 열기"""
    try:
        # a.grdTit 링크 클릭
        title_link = tr.find_element(By.CSS_SELECTOR, "a.grdTit")
        
        # onclick 속성에서 atcl_seq 추출
        onclick_attr = title_link.get_attribute("onclick") or ""
        # "brdAtclList.showGo(this,4099032,true)" → 4099032 추출
        atcl_seq_match = re.search(r'showGo\([^,]+,(\d+)', onclick_attr)
        if atcl_seq_match:
            atcl_seq = atcl_seq_match.group(1)
            print(f"  📌 게시글 ID: {atcl_seq}")
        
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", title_link)
        driver.execute_script("arguments[0].click();", title_link)
        time.sleep(2)
        return True
    except Exception as e:
        print(f"  ❌ 클릭 실패: {e}")
        try:
            # 더블클릭 시도
            ActionChains(driver).double_click(tr).perform()
            time.sleep(1.5)
            return True
        except:
            return False

def extract_row_info(tr):
    """목록 행에서 기본 정보 추출"""
    try:
        td = tr.find_element(By.CSS_SELECTOR, "td:has(a.grdTit)")
        
        # 게시판명 추출 (예: [전사게시판])
        board_name_el = td.find_elements(By.CSS_SELECTOR, "span[style*='color:rgb(83,193,75)']")
        board_name = board_name_el[0].text.strip() if board_name_el else ""
        
        # 제목 추출
        title_link = td.find_element(By.CSS_SELECTOR, "a.grdTit")
        title = norm_space(title_link.text)
        
        # onclick에서 atcl_seq 추출
        onclick_attr = title_link.get_attribute("onclick") or ""
        atcl_seq_match = re.search(r'showGo\([^,]+,(\d+)', onclick_attr)
        atcl_seq = atcl_seq_match.group(1) if atcl_seq_match else ""
        
        # NEW 아이콘 확인
        has_new = len(td.find_elements(By.CSS_SELECTOR, "span.o-i-new")) > 0
        
        # 댓글 수 추출
        comment_count_el = td.find_elements(By.CSS_SELECTOR, "span.F_11_gray[style*='color:red']")
        comment_count = norm_space(comment_count_el[0].text) if comment_count_el else "0"
        
        return {
            "atcl_seq": atcl_seq,
            "board_name": board_name,
            "title": title,
            "has_new": has_new,
            "comment_count": comment_count
        }
    except Exception as e:
        print(f"  ⚠️ 목록 정보 추출 실패: {e}")
        return None

def back_to_board_list():
    """상세에서 목록으로 복귀"""
    try:
        driver.switch_to.default_content()
        
        # content_frame으로 이동
        try:
            driver.switch_to.frame("content_frame")
        except:
            pass
        
        # 목록 버튼 클릭
        list_btn = driver.find_elements(By.XPATH, 
            "//button[contains(text(),'목록')] | //a[contains(text(),'목록')] | //span[contains(text(),'목록')]")
        if list_btn:
            driver.execute_script("arguments[0].click();", list_btn[0])
            time.sleep(1.5)
        
        driver.switch_to.default_content()
        return True
    except Exception as e:
        print(f"⚠️ 목록 복귀 실패: {e}")
        driver.switch_to.default_content()
        return False

# ─────────────────────────────────────────────────────────
# 6) 메인 실행 로직
# ─────────────────────────────────────────────────────────

# 브라우저 시작
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-popup-blocking")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(service=Service(), options=opts)

# 로그인
driver.get(BASE)
time.sleep(1.2)
input("로그인(필요시 '다음에 변경')까지 완료 후 Enter... ")

# 사용자 매핑 로드
print("📂 사용자 정보 로딩 중...")
user_map = load_user_mapping("users_filtered.csv")
print(f"✅ {len(user_map)}명의 사용자 정보 로드 완료")

# 게시판 메뉴 클릭
print("📋 게시판 메뉴 클릭 중...")
click_in_all_frames(
    css_list=["a[href='/brd/mywork/'].left_menu", "a[href='/brd/mywork/']"],
    xp_list=["//a[@href='/brd/mywork/']"],
    text_contains="게시판"
)
time.sleep(1.5)

# 게시판 목록 수집 (여기서는 특정 게시판 선택이 필요할 수 있음)
# 실제 화면 구조에 맞게 수정 필요
print("📊 게시판 목록 로딩 중...")
time.sleep(2)

# content_frame으로 전환
try:
    driver.switch_to.default_content()
    driver.switch_to.frame("content_frame")
except:
    pass

# 게시판 목록 수집
all_posts = []
all_comments = []
page_num = 1
max_pages = 999  # 테스트용
max_test = 50   # 페이지당 최대 수집 수

# 그리드 테이블 찾기 (a.grdTit가 있는 테이블)
GRID_SELECTOR = "table"  # 먼저 table을 찾고 a.grdTit 확인

while page_num <= max_pages:
    print(f"\n{'='*70}")
    print(f"📄 페이지 {page_num} 수집 중...")
    print(f"{'='*70}")
    
    try:
        # 프레임 재진입
        driver.switch_to.default_content()
        driver.switch_to.frame("content_frame")
        
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a.grdTit"))
        )
        
        # a.grdTit가 있는 행만 수집
        rows = driver.find_elements(By.CSS_SELECTOR, "tr:has(a.grdTit)")
        
        if not rows:
            # :has() 미지원 브라우저 대응
            all_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr, tr")
            rows = [r for r in all_rows if r.find_elements(By.CSS_SELECTOR, "a.grdTit")]
        
        print(f"  ✅ 게시글 {len(rows)}개 발견")
        
        # 페이지당 일부만 수집
        for idx in range(min(max_test, len(rows))):
            try:
                # 매번 rows 재조회 (Stale Element 방지)
                driver.switch_to.default_content()
                driver.switch_to.frame("content_frame")
                
                # a.grdTit가 있는 행만 수집
                current_rows = driver.find_elements(By.CSS_SELECTOR, "tr:has(a.grdTit)")
                if not current_rows:
                    all_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr, tr")
                    current_rows = [r for r in all_rows if r.find_elements(By.CSS_SELECTOR, "a.grdTit")]
                
                if idx >= len(current_rows):
                    print(f"⚠️ 행이 부족합니다 ({idx+1} > {len(current_rows)})")
                    break
                
                tr = current_rows[idx]
                
                # 목록 정보 추출
                row_info = extract_row_info(tr)
                if row_info:
                    print(f"\n📄 [{idx+1}/{min(max_test, len(rows))}] {row_info['title'][:50]}")
                    print(f"   게시판: {row_info['board_name']}, 댓글: {row_info['comment_count']}개")
                else:
                    print(f"\n📄 [{idx+1}/{min(max_test, len(rows))}] 처리 중...")
                
                # 행 클릭하여 상세 열기
                if not click_board_row(tr):
                    print("❌ 상세 열기 실패")
                    continue
                
                # 상세 파싱
                detail_data = parse_board_detail(driver, user_map)
                
                if not detail_data:
                    print("❌ 상세 파싱 실패")
                    back_to_board_list()
                    continue
                
                all_posts.append(detail_data["post"])
                all_comments.extend(detail_data["comments"])
                
                print(f"✅ 수집 완료: {detail_data['post']['title'][:50]}")
                print(f"   - 댓글 {len(detail_data['comments'])}개")
                
                # 목록 복귀
                back_to_board_list()
                time.sleep(1)
                
                # 프레임 재진입 (목록으로 돌아온 후)
                driver.switch_to.default_content()
                driver.switch_to.frame("content_frame")
                
                # 그리드 재로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.grdTit"))
                )
                
            except Exception as e:
                print(f"❌ 수집 실패: {e}")
                traceback.print_exc()
                back_to_board_list()
                continue
        
        print(f"✅ 페이지 {page_num} 완료: {min(max_test, len(rows))}건 시도")
        
        # 다음 페이지로 이동
        if has_next_page_board():
            if go_to_next_page_board():
                page_num += 1
                time.sleep(1)
            else:
                break
        else:
            print(f"✅ 마지막 페이지 도달")
            break
            
    except Exception as e:
        print(f"❌ 페이지 수집 실패: {e}")
        traceback.print_exc()
        break

# ─────────────────────────────────────────────────────────
# 7) 결과 저장
# ─────────────────────────────────────────────────────────
print(f"\n{'='*70}")
print(f"📊 수집 완료")
print(f"{'='*70}")
print(f"✅ 게시글: {len(all_posts)}건")
print(f"✅ 댓글: {len(all_comments)}건")

# JSON 저장
posts_json = {"posts": all_posts}
comments_json = {"comments": all_comments}

with open("board_posts.json", "w", encoding="utf-8") as f:
    json.dump(posts_json, f, ensure_ascii=False, indent=2)

with open("board_comments.json", "w", encoding="utf-8") as f:
    json.dump(comments_json, f, ensure_ascii=False, indent=2)

print("✅ JSON 저장 완료: board_posts.json, board_comments.json")

# DataFrame 변환 (DB 저장용)
if all_posts:
    df_posts = pd.DataFrame(all_posts)
    df_posts['writer_emailId'] = df_posts['writer'].apply(lambda x: x.get('emailId', ''))
    df_posts['attaches_json'] = df_posts['attaches'].apply(lambda x: json.dumps(x, ensure_ascii=False))
    df_posts = df_posts.drop(['writer', 'attaches'], axis=1)
    
    # Excel 저장
    df_posts.to_excel("게시판_목록.xlsx", index=False, engine="openpyxl")
    df_posts.to_csv("게시판_목록.csv", index=False, encoding="utf-8-sig")
    print("✅ 파일 저장 완료: 게시판_목록.xlsx / 게시판_목록.csv")
    
    # DB 저장
    USER = "root"
    PASS = "1234"
    HOST = "127.0.0.1"
    PORT = 3306
    DB = "crawling"
    
    engine = create_engine(f"mysql+pymysql://{USER}:{PASS}@{HOST}:{PORT}/{DB}?charset=utf8mb4")
    
    df_posts.to_sql(
        name="board_posts",
        con=engine,
        if_exists="replace",
        index=False,
        dtype={
            "sourceId": VARCHAR(100),
            "targetBoardId": VARCHAR(50),
            "createdAt": VARCHAR(30),
            "updatedAt": VARCHAR(30),
            "inlineHandling": VARCHAR(10),
            "writer_emailId": VARCHAR(100),
            "title": VARCHAR(500),
            "readCount": VARCHAR(20),
            "content": LONGTEXT(),
            "status": VARCHAR(20),
            "attaches_json": TEXT()
        }
    )
    print("✅ DB 저장 완료 (테이블: board_posts)")

if all_comments:
    df_comments = pd.DataFrame(all_comments)
    df_comments['writer_emailId'] = df_comments['writer'].apply(lambda x: x.get('emailId', ''))
    df_comments = df_comments.drop(['writer'], axis=1)
    
    df_comments.to_excel("게시판_댓글.xlsx", index=False, engine="openpyxl")
    df_comments.to_csv("게시판_댓글.csv", index=False, encoding="utf-8-sig")
    print("✅ 파일 저장 완료: 게시판_댓글.xlsx / 게시판_댓글.csv")
    
    df_comments.to_sql(
        name="board_comments",
        con=engine,
        if_exists="replace",
        index=False,
        dtype={
            "sourceId": VARCHAR(100),
            "sourcePostId": VARCHAR(100),
            "createdAt": VARCHAR(30),
            "writer_emailId": VARCHAR(100),
            "message": TEXT(),
            "attaches": VARCHAR(500)
        }
    )
    print("✅ DB 저장 완료 (테이블: board_comments)")

# driver.quit()
print("\n🎉 모든 작업 완료!")



로그인(필요시 '다음에 변경')까지 완료 후 Enter...  


📂 사용자 정보 로딩 중...
✅ 184명의 사용자 정보 로드 완료
📋 게시판 메뉴 클릭 중...
📊 게시판 목록 로딩 중...

📄 페이지 1 수집 중...
  ✅ 게시글 50개 발견

📄 [1/50] 제1회 대한민국 인공지능 혁신대상- “종합대상” 수상
   게시판: [전사게시판], 댓글: 2개
  📌 게시글 ID: 4102705
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 제1회 대한민국 인공지능 혁신대상- “종합대상” 수상
   - 댓글 2개

📄 [2/50] [부고] 박종아선임 외조부상
   게시판: [전사게시판], 댓글: 1개
  📌 게시글 ID: 4102468
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: [부고] 박종아선임 외조부상
   - 댓글 1개

📄 [3/50] [부고] 황혜림선임 조모상
   게시판: [전사게시판], 댓글: 0개
  📌 게시글 ID: 4099507
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: [부고] 황혜림선임 조모상
   - 댓글 1개

📄 [4/50] (장기 근속 축하) 송준선 상무님 10년 근속을 축하 드립니다
   게시판: [전사게시판], 댓글: 0개
  📌 게시글 ID: 4099032
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: (장기 근속 축하) 송준선 상무님 10년 근속을 축하 드립니다
   - 댓글 4개

📄 [5/50] 신입사원 입사안내!! (황유성 사원)
   게시판: [전사게시판], 댓글: 0개
  📌 게시글 ID: 4098797
  ✓ content_frame 전환 완료
  ✓ 상세 페이지 로딩 완료
✅ 수집 완료: 신입사원 입사안내!! (황유성 사원)
   - 댓글 1개

📄 [6/50] 11월의 생일자들을 알려드려요. 다같이 축하해 주세요~
   게시판: [사.우.회], 댓글: 0개
  📌 게시글 ID: 4098793
  

KeyboardInterrupt: 